# Deploy ML models to SageMaker with MLflow 

## What you'll learn in this course 

Serving models locally is fine but they won't be available once you'll turn your computer off. In this course, you'll learn:

* Set up your aws cli to prepare for deployment 
* Build Push your docker image to ECR 
* Create a SageMaker API endpoint and use it 

## Prepare your environment ☄️☄️

Before diving into mlflow, you will need to setup up your environment to make sure everything is correct. Here is the process: 

1. Make sure you have Docker installed 
2. Make sure boto3 is installed: `pip install boto3`
3. install `aws cli` and set a region and default credentials
4. Log in to ecr and register your container url to Docker
5. Make sure your `conda.yml` file has a version of sklearn that is not set. Especially `sklearn=0.23.xx` does not work

### Have Boto3 installed 

Simply run `pip install boto3`

### Install AWS cli 

Now that you need to install aws cli to setup a region and default credential. To install aws cli, follow this link: 

* [aws cli linux](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-linux.html)
* [aws cli for mac](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-mac.html)
* [aws cli for windows](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-windows.html)


Once you have it installed, you can go to your terminal and run:

```shell 
aws --version
```

You should get the following output: 

```shell 
aws-cli/2.0.22 Python/3.7.4 Darwin/19.5.0 botocore/2.0.0dev26
```

### Setup aws creds & region 

For mlflow to work, you will need to setup aws credentials and region. Here are the commands: 

* `aws configure set aws_access_key_id ACCESS_KEY`
* `aws configure set aws_secret_access_key SECRET_KEY`
* `aws configure set default.region REGION`

Replace with your credentials and region you want your model to be deploy. 

### Create an ECR registry 

Now go to AWS and create an ECR registry. Once you are done, you should be able to click on `view push commands` 


![](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/view-push-command-ecr.png)



Then run ONLY THE FIRST command on your terminal 

![](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/login-ecr-docker.png)

### Verify your `conda.yaml` file 

One final thing is to verify your `conda.yaml` file. For example, `sklearn=0.23.1` doesn't work. You should instead have `sklearn` instead. 

## Create an execution role 

If you want your the deployment to work, you will need to create a role with AMAZON_SAGEMAKER_FULL_ACCESS 

![](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/ecr_execution_role.png)

Copy the ARN as you will need to use it later on for deployment. 

## Deploy to SageMaker 

Now that everything is prepared, you can go to your project and add the following code:

In [ ]:
# Region where you want your model to be deployed
region = "REGION_NAME"

# Get it from Mlflow 
run_id = "MLFLOW_RUN_ID"
model_uri = "runs:/" + run_id + "/model"
execution_role="ARN_ROLE_FOR_SAGEMAKER"
instance_type="INSTANCE_TYPE"

# Get it directly from AWS
image_ecr_url = "ECR_IMAGE_URL"

Every variables should now be easy to fill out except `instance_type`. You can choose all the instance type from this list: 

* [Supported Instance Type for a given region](https://docs.aws.amazon.com/sagemaker/latest/dg/instance-types-az.html)

NB: Check the [AWS Free tier](https://aws.amazon.com/fr/free/?all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc), if you want to make you won't be charged for using instances. 

Now finally we can run the following command to deploy your ML model to SageMaker

In [ ]:
import mlflow.sagemaker as mfs
# Define an app
app_name = "iris"
mfs.deploy(
        app_name=app_name, 
        model_uri=model_uri, 
        image_url=image_ecr_url, 
        region_name=region, 
        mode="replace", 
        execution_role_arn=execution_role,
        instance_type=instance_type)

## Use your deployed model 

Now you can use your deploy model! You first need to prepare your data the following way:

In [ ]:
# Format input for API 
input_json = X_test.to_json(orient="split")

And let's create the following function: 

In [ ]:
# Use the API endpoint
import json
import boto3

def query_endpoint(app_name, input_json):

    client = boto3.session.Session().client("sagemaker-runtime", region)
    
    response = client.invoke_endpoint(
        EndpointName=app_name,
        Body=input_json,
        ContentType='application/json; format=pandas-split',
    )
    preds = response['Body'].read().decode("ascii")
    preds = json.loads(preds)
    print("Received response: {}".format(preds))

    return preds

Use the function: 

In [ ]:
# Evaluate the input by posting it to the deployed model
prediction1 = query_endpoint(app_name=app_name, input_json=input_json)

## Delete the app 

To make you are not overcharged, delete your app

In [ ]:
# Delete app 
import mlflow.sagemaker as mfs
app_name = "iris"
# Specify the archive=False option to delete any SageMaker models and configurations
# associated with the specified application
mfs.delete(app_name=app_name, region_name=region, archive=False)

## Resources 

* [Set up AWS authentication for SageMaker deployment](https://docs.databricks.com/administration-guide/cloud-configurations/aws/sagemaker.html)

* [MLflow Quick Start Part 2: Serving Models via Amazon SageMaker](https://docs.databricks.com/_static/notebooks/mlflow/mlflow-quick-start-deployment-aws.html)

* [Installing the AWS CLI version 2](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html)

* [Installing the AWS CLI version 2 on Windows](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-windows.html)

* [Docker Desktop overview](https://docs.docker.com/desktop/)

* [Offre gratuite d'AWS](https://aws.amazon.com/fr/free/?all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc)

* [Supported Instance Types and Availability Zones](https://docs.aws.amazon.com/sagemaker/latest/dg/instance-types-az.html)